In [ ]:
##import libs
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
import glob
from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix 
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.manifold import TSNE
from colour import Color
from sklearn.covariance import EllipticEnvelope
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.datasets import make_moons, make_blobs
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix 
from psynlig import plot_correlation_heatmap
import pingouin as pg
from scipy.stats import pearsonr
import plotly.express as px
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import chart_studio as cs
import numpy as np
import pandas as pd
import scipy
import chart_studio.plotly as py
from plotly.offline import iplot
import pandas as pd
import numpy as np  
%matplotlib inline  
from plotly import __version__ 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
import chart_studio.plotly as py
from sklearn import preprocessing
from pandas.plotting import scatter_matrix 
from psynlig import plot_correlation_heatmap
import pingouin as pg
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler

In [ ]:
##functions

In [ ]:
def summarize_dataframe(df, group_col):

    grouped_df = df.groupby(group_col)
   

    agg_dict = {}
   

    for col in df.columns:
        if col != group_col:
            agg_dict[col] = [
                ('count', 'count'),
                ('mean', 'mean'),
                ('std', 'std'),
                ('min', 'min'),
                ('25%', lambda x: np.quantile(x, 0.25)),
                ('median', 'median'),
                ('75%', lambda x: np.quantile(x, 0.75)),
                ('max', 'max')
                
            ]
   

    summary_df = grouped_df.agg(agg_dict)
   

    summary_df.columns = [f'{col}_{stat}' for col, stat in summary_df.columns]
   

    summary_df.index.name = group_col
   
    return summary_df

In [ ]:
def add_plate_column(folder_path):
    files = os.listdir(folder_path)
    output_folder = os.path.join(folder_path, "output")
    os.makedirs(output_folder, exist_ok=True)
   
    for i, file in enumerate(files):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            output_path = os.path.join(output_folder, file)
           
            df = pd.read_csv(file_path)
            df.insert(0, "plate", f"p{i+1}")
            df['Row'] = 'r' + df['Row'].astype(str)
            df['Column'] = 'c' + df['Column'].astype(str)
            df["pos"] = df["Row"].astype(str) + df["Column"].astype(str)  + df["plate"].astype(str) 
            df.to_csv(output_path, index=False)

In [ ]:
def add_exp_column_to_csv_files(input_path, output_path):

    if not os.path.exists(output_path):
        os.makedirs(output_path)
   

    csv_files = [file for file in os.listdir(input_path) if file.endswith('.csv')]
   
    for file in csv_files:

        file_path = os.path.join(input_path, file)
        df = pd.read_csv(file_path)
       

        exp_value = file[7:10] 
       

        df['EXP'] = exp_value
       

        output_file_path = os.path.join(output_path, file)
        df.to_csv(output_file_path, index=False)

In [ ]:
path = ('RESULTS_mito_markers_feat_seelction') 
if not os.path.exists(path):
    os.mkdir(path)
    print('Output folder created')

In [ ]:
#DRP1

In [ ]:
all_files = glob.glob('noam_may_analysis_fold1/DRP1/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

drp1_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
#remove treated data for feat select
cell_types =  drp1_df["Compound"].values
sns.histplot(cell_types)
plt.show()
print(f'Total number of cell types {len(cell_types)}')
drp1_df = drp1_df.loc[cell_types != 'mdivi1 25uM']
sns.histplot(drp1_df["Compound"].values)
plt.show()
print(f'Total number of cells {len(drp1_df)}')

In [ ]:
##define groups 
df_h = drp1_df[(drp1_df["Severity"] == 'HC')] 
df_s = drp1_df[(drp1_df["Severity"] == 'S')]
frame_drp = (df_h, df_s)
frame_drp_f = pd.concat(frame_drp)

In [45]:
HEAT_DATA_drp1 = frame_drp_f.copy()
HEAT_DATA_drp1["group_with_id"] = HEAT_DATA_drp1["Cell ID"].astype(str) + HEAT_DATA_drp1["Cell Type"].astype(str) 
HEAT_DATA_drp1.set_index(["group_with_id"], inplace = True,
                            append = True, drop = False)
CLEAN_dfheatmapREAL_drp = HEAT_DATA_drp1.groupby(level=1).mean()

CLEAN_dfheatmap_final_drp = CLEAN_dfheatmapREAL_drp.drop(columns=['Row', 'Plane', 'Timepoint', 'Number of Analyzed Fields', 'Height [µm]',
                            'Time [s]', 'Column', "Cell ID"])
CLEAN_dfheatmap_final_drp.fillna(0, inplace=True)
CLEAN_dfheatmap_final_NORMAL_drp = CLEAN_dfheatmap_final_drp.copy()
CLEAN_dfheatmap_final_NORMAL_drp=(CLEAN_dfheatmap_final_NORMAL_drp-CLEAN_dfheatmap_final_NORMAL_drp.mean())/CLEAN_dfheatmap_final_NORMAL_drp.std()

C:\Users\MiguelW12\AppData\Local\Temp\ipykernel_9812\3460966092.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  CLEAN_dfheatmapREAL_drp = HEAT_DATA_drp1.groupby(level=1).mean()


In [46]:
file1 = 'mito_drp1_cor.csv'
CLEAN_dfheatmap_final_NORMAL_drp.to_csv(file1)

In [ ]:
frame_drp_f = frame_drp_f.drop(columns=['Row', 'Plane', 'Timepoint', 'Number of Analyzed Fields', 'Height [µm]',
                            'Time [s]', 'Column', 'Compound', "Cell Type", "Cell ID"])

frame_drp_f = frame_drp_f.rename(columns={"Severity":"group"})

In [50]:
##sum data
frame_drp_f = frame_drp_f.dropna()
summary_df = summarize_dataframe(frame_drp_f,'group')
file2 = 'summary_mito_drp.csv'
summary_df.to_csv(file2)

In [51]:
file3 = 'mito_drp1_combined.csv'
frame_drp_f.to_csv(file3)

In [ ]:
##PCA
PCA_DATA = frame_drp_f.copy()
PCA_DATA.fillna(0, inplace=True)
features2 = PCA_DATA.columns[:54]  
x2 = PCA_DATA.loc[:, features2].values
y2 = PCA_DATA.loc[:,['group']].values
x2 = StandardScaler().fit_transform(x2)
pd.DataFrame(data = x2, columns = features2).head()
pca2 = PCA(n_components=2)
principalComponents2 = pca2.fit_transform(x2)
PCA_DATA2_COMP = pd.DataFrame(data = principalComponents2
             , columns = ['principal component 1', 'principal component 2'])
importances2 = pd.DataFrame(data = abs(pca2.components_).transpose(), columns = ['PC1', 'PC2'])
importances2['Feature'] = features2
display(importances2.sort_values('PC1', ascending = False).reset_index(drop = True))
imp2 = importances2.sort_values('PC2', ascending = False)[0:30]

ax2 = sns.barplot(x = 'PC2', y = 'Feature', data = imp2)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90)  
plt.show()
    
imp4 = importances2.sort_values('PC1', ascending = False)[0:30]
ax2 = sns.barplot(x = 'PC1', y = 'Feature', data = imp4)

fig = ax2.get_figure()
fig.set_size_inches(4, 8)
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90)  
plt.show()
    
PCA_DATA2_FINAL = pd.concat([PCA_DATA2_COMP, PCA_DATA[['group']]], axis = 1)
PCA_FIG2 = plt.figure(figsize = (8,8))
ax2 = PCA_FIG2.add_subplot(1,1,1) 
ax2.set_xlabel('Principal Component 1', fontsize = 15)
ax2.set_ylabel('Principal Component 2', fontsize = 15)
ax2.set_title('2 Component PCA', fontsize = 20)
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)
for tick in ax2.get_xticklabels():
    tick.set_fontname("Arial")
for tick in ax2.get_yticklabels():
    tick.set_fontname("Arial")
    
    
    

targets2 = ['HC', 'S']
colors2 = ['b', 'r']
for target, color in zip(targets2,colors2):
    indicesToKeep = PCA_DATA2_FINAL['group'] == target
    ax2.scatter(PCA_DATA2_FINAL.loc[indicesToKeep, 'principal component 1']
               , PCA_DATA2_FINAL.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
    ax2.legend(targets2)
    ax2.grid()

print(pca2.explained_variance_ratio_)
plt.savefig(path + '//' + 'drp1_PCA2.pdf', dpi = 300)
plt.show() 

In [ ]:
#LDA
PCA_DATA_CLEAN = frame_drp_f.copy()

x = PCA_DATA_CLEAN.loc[:, features2].values
y = PCA_DATA_CLEAN.loc[:,['group']].values
y = np.array([int('HC' == y_i[0][:2]) for y_i in y])

y[0]=2
x = StandardScaler().fit_transform(x)

lda = LDA(n_components=2)
lda_x = np.array(lda.fit(x, y).transform(x))


importances = pd.DataFrame(data = abs(lda.scalings_), columns = ['PC1', 'PC2'])
importances['Feature'] = features2
display(importances.sort_values('PC1', ascending = False).reset_index(drop = True))
imp = importances.sort_values('PC2', ascending = False)[0:30]
ax1 = sns.barplot(x = 'PC2', y = 'Feature', data = imp)
    
fig = ax1.get_figure()
fig.set_size_inches(4, 8)
plt.rc('ytick', labelsize=10)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation = 90)  
plt.show()
imp2 = importances.sort_values('PC1', ascending = False)[0:30]
ax2 = sns.barplot(x = 'PC1', y = 'Feature', data = imp2)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
plt.rc('ytick', labelsize=10)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90) 
plt.show()

font = {'family' : 'Arial',
        'size'   : 22}

plt.rc('font', **font)

scale=6
target_names= ['S', 'HC']
plt.figure()
plt.figure(figsize = (8,8))
for c, i, target_name in zip("rb", [0, 1], target_names):
    plt.scatter(lda_x[y == i,0] , lda_x[y == i,1] , c=c, label=target_name)
    plt.legend(target_names*scale,fontsize=12)
    plt.xlabel('LD1', fontsize = 20, family = 'Arial')
    plt.ylabel('LD2', fontsize = 20, family = 'Arial')
    plt.yticks(family = 'Arial', size = 10)
    plt.xticks(family = 'Arial', size= 10)
print(lda.explained_variance_ratio_)            
plt.savefig(path + '\\' + 'drp-LDA.pdf', dpi=600 )
plt.show()

In [ ]:
#VAT1

In [ ]:
all_files = glob.glob('noam_may_analysis_fold1/VAT1/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

vat1_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
cell_types =  vat1_df["Compound"].values
sns.histplot(cell_types)
plt.show()
print(f'Total number of cell types {len(cell_types)}')
vat1_df = vat1_df.loc[cell_types != 'mdivi1 25uM']
sns.histplot(vat1_df["Compound"].values)
plt.show()
print(f'Total number of cells {len(vat1_df)}')

In [ ]:
df_h = vat1_df[(vat1_df["Severity"] == 'HC')] 
df_s = vat1_df[(vat1_df["Severity"] == 'S')]
frame_vat = (df_h, df_s)
frame_vat_f = pd.concat(frame_vat)

In [57]:
HEAT_DATA_vat1 = frame_vat_f.copy()
HEAT_DATA_vat1["group_with_id"] = HEAT_DATA_vat1["Cell ID"].astype(str) + HEAT_DATA_vat1["Cell Type"].astype(str) 
HEAT_DATA_vat1.set_index(["group_with_id"], inplace = True,
                            append = True, drop = False)
CLEAN_dfheatmapREAL_vat1 = HEAT_DATA_vat1.groupby(level=1).mean()

CLEAN_dfheatmap_final_vat1 = CLEAN_dfheatmapREAL_vat1.drop(columns=['Row', 'Plane', 'Timepoint', 'Number of Analyzed Fields', 'Height [µm]',
                            'Time [s]', 'Column', "Cell ID"])
CLEAN_dfheatmap_final_vat1.fillna(0, inplace=True)
CLEAN_dfheatmap_final_NORMAL_vat1 = CLEAN_dfheatmap_final_vat1.copy()
CLEAN_dfheatmap_final_NORMAL_vat1=(CLEAN_dfheatmap_final_NORMAL_vat1-CLEAN_dfheatmap_final_NORMAL_vat1.mean())/CLEAN_dfheatmap_final_NORMAL_vat1.std()

C:\Users\MiguelW12\AppData\Local\Temp\ipykernel_9812\2454812790.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  CLEAN_dfheatmapREAL_vat1 = HEAT_DATA_vat1.groupby(level=1).mean()


In [58]:
file1 = 'mito_vat1_cor.csv'
CLEAN_dfheatmap_final_NORMAL_vat1.to_csv(file1)

In [ ]:
frame_vat_f = frame_vat_f.drop(columns=['Row', 'Plane', 'Timepoint', 'Number of Analyzed Fields', 'Height [µm]',
                            'Time [s]', 'Column', 'Compound', "Cell Type", "Cell ID"])

frame_vat_f = frame_vat_f.rename(columns={"Severity":"group"})

In [63]:
frame_vat_f = frame_vat_f.dropna()
summary_df = summarize_dataframe(frame_vat_f,'group')
file1 = 'summary_mito_vat.csv'
summary_df.to_csv(file1)

In [64]:
file1 = 'mito_vat1_combined.csv'
frame_vat_f.to_csv(file1)

In [ ]:
PCA_DATA = frame_vat_f.copy()
PCA_DATA.fillna(0, inplace=True)
features2 = PCA_DATA.columns[:54]  
x2 = PCA_DATA.loc[:, features2].values
y2 = PCA_DATA.loc[:,['group']].values
x2 = StandardScaler().fit_transform(x2)
pd.DataFrame(data = x2, columns = features2).head()
pca2 = PCA(n_components=2)
principalComponents2 = pca2.fit_transform(x2)
PCA_DATA2_COMP = pd.DataFrame(data = principalComponents2
             , columns = ['principal component 1', 'principal component 2'])
importances2 = pd.DataFrame(data = abs(pca2.components_).transpose(), columns = ['PC1', 'PC2'])
importances2['Feature'] = features2
display(importances2.sort_values('PC1', ascending = False).reset_index(drop = True))
imp2 = importances2.sort_values('PC2', ascending = False)[0:30]

ax2 = sns.barplot(x = 'PC2', y = 'Feature', data = imp2)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90)  
plt.show()
    
imp4 = importances2.sort_values('PC1', ascending = False)[0:30]
ax2 = sns.barplot(x = 'PC1', y = 'Feature', data = imp4)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90)  
plt.show()
    
PCA_DATA2_FINAL = pd.concat([PCA_DATA2_COMP, PCA_DATA[['group']]], axis = 1)
PCA_FIG2 = plt.figure(figsize = (8,8))
ax2 = PCA_FIG2.add_subplot(1,1,1) 
ax2.set_xlabel('Principal Component 1', fontsize = 15)
ax2.set_ylabel('Principal Component 2', fontsize = 15)
ax2.set_title('2 Component PCA', fontsize = 20)
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)
for tick in ax2.get_xticklabels():
    tick.set_fontname("Arial")
for tick in ax2.get_yticklabels():
    tick.set_fontname("Arial")
    
    
    

targets2 = ['HC', 'S']
colors2 = ['b', 'r']
for target, color in zip(targets2,colors2):
    indicesToKeep = PCA_DATA2_FINAL['group'] == target
    ax2.scatter(PCA_DATA2_FINAL.loc[indicesToKeep, 'principal component 1']
               , PCA_DATA2_FINAL.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
    ax2.legend(targets2)
    ax2.grid()

    
print(pca2.explained_variance_ratio_)
plt.savefig(path + '//' + 'vat1_PCA2.pdf', dpi = 300)
plt.show()

In [ ]:
##LDA

PCA_DATA_CLEAN = frame_vat_f.copy()

x = PCA_DATA_CLEAN.loc[:, features2].values
y = PCA_DATA_CLEAN.loc[:,['group']].values
y = np.array([int('HC' == y_i[0][:2]) for y_i in y])

y[0]=2
x = StandardScaler().fit_transform(x)

lda = LDA(n_components=2)
lda_x = np.array(lda.fit(x, y).transform(x))


importances = pd.DataFrame(data = abs(lda.scalings_), columns = ['PC1', 'PC2'])
importances['Feature'] = features2
display(importances.sort_values('PC1', ascending = False).reset_index(drop = True))
imp = importances.sort_values('PC2', ascending = False)[0:30]
ax1 = sns.barplot(x = 'PC2', y = 'Feature', data = imp)
    
fig = ax1.get_figure()
fig.set_size_inches(4, 8)
plt.rc('ytick', labelsize=10)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation = 90)  
plt.show()
imp2 = importances.sort_values('PC1', ascending = False)[0:30]
ax2 = sns.barplot(x = 'PC1', y = 'Feature', data = imp2)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
plt.rc('ytick', labelsize=10)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90) 
plt.show()

font = {'family' : 'Arial',
        'size'   : 22}

plt.rc('font', **font)

scale=6
target_names= ['S', 'HC']
plt.figure()
plt.figure(figsize = (8,8))
for c, i, target_name in zip("rb", [0, 1], target_names):
    plt.scatter(lda_x[y == i,0] , lda_x[y == i,1] , c=c, label=target_name)
    plt.legend(target_names*scale,fontsize=12)
    plt.xlabel('LD1', fontsize = 20, family = 'Arial')
    plt.ylabel('LD2', fontsize = 20, family = 'Arial')
    plt.yticks(family = 'Arial', size = 10)
    plt.xticks(family = 'Arial', size= 10)
print(lda.explained_variance_ratio_)            
plt.savefig(path + '\\' + 'vat-LDA.pdf', dpi=600 )
plt.show()

In [ ]:
#MFN1

In [ ]:
all_files = glob.glob('noam_may_analysis_fold1/MFN1/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

mfn1_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
cell_types =  mfn1_df["Compound"].values
sns.histplot(cell_types)
plt.show()
print(f'Total number of cell types {len(cell_types)}')
mfn1_df = mfn1_df.loc[cell_types != 'mdivi1 25uM']
sns.histplot(mfn1_df["Compound"].values)
plt.show()
print(f'Total number of cells {len(mfn1_df)}')

In [ ]:
df_h = mfn1_df[(mfn1_df["Severity"] == 'HC')] 
df_s = mfn1_df[(mfn1_df["Severity"] == 'S')]
frame_mfn1 = (df_h, df_s)
frame_mfn1_f = pd.concat(frame_mfn1)

In [70]:
HEAT_DATA_mfn1 = frame_mfn1_f.copy()
HEAT_DATA_mfn1["group_with_id"] = HEAT_DATA_mfn1["Cell ID"].astype(str) + HEAT_DATA_mfn1["Cell Type"].astype(str) 
HEAT_DATA_mfn1.set_index(["group_with_id"], inplace = True,
                            append = True, drop = False)
CLEAN_dfheatmapREAL_mfn1 = HEAT_DATA_mfn1.groupby(level=1).mean()

CLEAN_dfheatmap_final_mfn1 = CLEAN_dfheatmapREAL_mfn1.drop(columns=['Row', 'Plane', 'Timepoint', 'Number of Analyzed Fields', 'Height [µm]',
                            'Time [s]', 'Column', "Cell ID"])
CLEAN_dfheatmap_final_mfn1.fillna(0, inplace=True)
CLEAN_dfheatmap_final_NORMAL_mfn1 = CLEAN_dfheatmap_final_mfn1.copy()
CLEAN_dfheatmap_final_NORMAL_mfn1=(CLEAN_dfheatmap_final_NORMAL_mfn1-CLEAN_dfheatmap_final_NORMAL_mfn1.mean())/CLEAN_dfheatmap_final_NORMAL_mfn1.std()

C:\Users\MiguelW12\AppData\Local\Temp\ipykernel_9812\1051816466.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  CLEAN_dfheatmapREAL_mfn1 = HEAT_DATA_mfn1.groupby(level=1).mean()


In [71]:
file1 = 'mito_mfn1_cor.csv'
CLEAN_dfheatmap_final_NORMAL_mfn1.to_csv(file1)

In [ ]:
frame_mfn1_f = frame_mfn1_f.drop(columns=['Row', 'Plane', 'Timepoint', 'Number of Analyzed Fields', 'Height [µm]',
                            'Time [s]', 'Column', 'Compound', "Cell Type", "Cell ID"])

frame_mfn1_f = frame_mfn1_f.rename(columns={"Severity":"group"})

In [75]:
frame_mfn1_f = frame_mfn1_f.dropna()
summary_df = summarize_dataframe(frame_mfn1_f,'group')
file1 = 'summary_mito_mfn1.csv'
summary_df.to_csv(file1)

In [76]:
file1 = 'mito_mfn1_combined.csv'
frame_mfn1_f.to_csv(file1)

In [ ]:
#PCA
PCA_DATA = frame_mfn1_f.copy()
PCA_DATA.fillna(0, inplace=True)
features2 = PCA_DATA.columns[:54]  
x2 = PCA_DATA.loc[:, features2].values
y2 = PCA_DATA.loc[:,['group']].values
x2 = StandardScaler().fit_transform(x2)
pd.DataFrame(data = x2, columns = features2).head()
pca2 = PCA(n_components=2)
principalComponents2 = pca2.fit_transform(x2)
PCA_DATA2_COMP = pd.DataFrame(data = principalComponents2
             , columns = ['principal component 1', 'principal component 2'])
importances2 = pd.DataFrame(data = abs(pca2.components_).transpose(), columns = ['PC1', 'PC2'])
importances2['Feature'] = features2
display(importances2.sort_values('PC1', ascending = False).reset_index(drop = True))
imp2 = importances2.sort_values('PC2', ascending = False)[0:30]

ax2 = sns.barplot(x = 'PC2', y = 'Feature', data = imp2)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90)  
plt.show()
    
imp4 = importances2.sort_values('PC1', ascending = False)[0:30]
ax2 = sns.barplot(x = 'PC1', y = 'Feature', data = imp4)

fig = ax2.get_figure()
fig.set_size_inches(4, 8)
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90)  
plt.show()
    
PCA_DATA2_FINAL = pd.concat([PCA_DATA2_COMP, PCA_DATA[['group']]], axis = 1)
PCA_FIG2 = plt.figure(figsize = (8,8))
ax2 = PCA_FIG2.add_subplot(1,1,1) 
ax2.set_xlabel('Principal Component 1', fontsize = 15)
ax2.set_ylabel('Principal Component 2', fontsize = 15)
ax2.set_title('2 Component PCA', fontsize = 20)
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)
for tick in ax2.get_xticklabels():
    tick.set_fontname("Arial")
for tick in ax2.get_yticklabels():
    tick.set_fontname("Arial")
    
    
    

targets2 = ['HC', 'S']
colors2 = ['b', 'r']
for target, color in zip(targets2,colors2):
    indicesToKeep = PCA_DATA2_FINAL['group'] == target
    ax2.scatter(PCA_DATA2_FINAL.loc[indicesToKeep, 'principal component 1']
               , PCA_DATA2_FINAL.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
    ax2.legend(targets2)
    ax2.grid()

    
print(pca2.explained_variance_ratio_)
plt.savefig(path + '//' + 'mfn1_PCA2.pdf', dpi = 300)
plt.show()
    
    

In [ ]:
#LDA
PCA_DATA_CLEAN = frame_mfn1_f.copy()

x = PCA_DATA_CLEAN.loc[:, features2].values
y = PCA_DATA_CLEAN.loc[:,['group']].values
y = np.array([int('HC' == y_i[0][:2]) for y_i in y])

y[0]=2
x = StandardScaler().fit_transform(x)

lda = LDA(n_components=2)
lda_x = np.array(lda.fit(x, y).transform(x))


importances = pd.DataFrame(data = abs(lda.scalings_), columns = ['PC1', 'PC2'])
importances['Feature'] = features2
display(importances.sort_values('PC1', ascending = False).reset_index(drop = True))
imp = importances.sort_values('PC2', ascending = False)[0:30]
ax1 = sns.barplot(x = 'PC2', y = 'Feature', data = imp)
    
fig = ax1.get_figure()
fig.set_size_inches(4, 8)
plt.rc('ytick', labelsize=10)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation = 90)  
plt.show()
imp2 = importances.sort_values('PC1', ascending = False)[0:30]
ax2 = sns.barplot(x = 'PC1', y = 'Feature', data = imp2)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
plt.rc('ytick', labelsize=10)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90) 
plt.show()

font = {'family' : 'Arial',
        'size'   : 22}

plt.rc('font', **font)

scale=6
target_names= ['S', 'HC']
plt.figure()
plt.figure(figsize = (8,8))
for c, i, target_name in zip("rb", [0, 1], target_names):
    plt.scatter(lda_x[y == i,0] , lda_x[y == i,1] , c=c, label=target_name)
    plt.legend(target_names*scale,fontsize=12)
    plt.xlabel('LD1', fontsize = 20, family = 'Arial')
    plt.ylabel('LD2', fontsize = 20, family = 'Arial')
    plt.yticks(family = 'Arial', size = 10)
    plt.xticks(family = 'Arial', size= 10)
print(lda.explained_variance_ratio_)            
plt.savefig(path + '\\' + 'mfn1-LDA.pdf', dpi=600 )
plt.show()

In [ ]:
#MFN2

In [ ]:
all_files = glob.glob('noam_may_analysis_fold1/MFN2/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

mfn2_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
cell_types =  mfn2_df["Compound"].values
sns.histplot(cell_types)
plt.show()
print(f'Total number of cell types {len(cell_types)}')
mfn2_df = mfn2_df.loc[cell_types != 'mdivi1 25uM']
sns.histplot(mfn2_df["Compound"].values)
plt.show()
print(f'Total number of cells {len(mfn2_df)}')

In [ ]:
df_h = mfn2_df[(mfn2_df["Severity"] == 'HC')] 
df_s = mfn2_df[(mfn2_df["Severity"] == 'S')]
frame_mfn2 = (df_h, df_s)
frame_mfn2_f = pd.concat(frame_mfn2)

In [84]:
HEAT_DATA_mfn2 = frame_mfn2_f.copy()
HEAT_DATA_mfn2["group_with_id"] = HEAT_DATA_mfn2["Cell ID"].astype(str) + HEAT_DATA_mfn2["Cell Type"].astype(str) 
HEAT_DATA_mfn2.set_index(["group_with_id"], inplace = True,
                            append = True, drop = False)
CLEAN_dfheatmapREAL_mfn2 = HEAT_DATA_mfn2.groupby(level=1).mean()

CLEAN_dfheatmap_final_mfn2 = CLEAN_dfheatmapREAL_mfn2.drop(columns=['Row', 'Plane', 'Timepoint', 'Number of Analyzed Fields', 'Height [µm]',
                            'Time [s]', 'Column', "Cell ID"])
CLEAN_dfheatmap_final_mfn2.fillna(0, inplace=True)
CLEAN_dfheatmap_final_NORMAL_mfn2 = CLEAN_dfheatmap_final_mfn2.copy()
CLEAN_dfheatmap_final_NORMAL_mfn2=(CLEAN_dfheatmap_final_NORMAL_mfn2-CLEAN_dfheatmap_final_NORMAL_mfn2.mean())/CLEAN_dfheatmap_final_NORMAL_mfn2.std()

C:\Users\MiguelW12\AppData\Local\Temp\ipykernel_9812\3976612670.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  CLEAN_dfheatmapREAL_mfn2 = HEAT_DATA_mfn2.groupby(level=1).mean()


In [85]:
file1 = 'mito_mfn2_cor.csv'
CLEAN_dfheatmap_final_NORMAL_mfn2.to_csv(file1)

In [ ]:
frame_mfn2_f = frame_mfn2_f.drop(columns=['Row', 'Plane', 'Timepoint', 'Number of Analyzed Fields', 'Height [µm]',
                            'Time [s]', 'Column', 'Compound', "Cell Type", "Cell ID"])

frame_mfn2_f = frame_mfn2_f.rename(columns={"Severity":"group"})

In [89]:
frame_mfn2_f = frame_mfn2_f.dropna()
summary_df = summarize_dataframe(frame_mfn2_f,'group')
file1 = 'summary_mito_mfn2.csv'
summary_df.to_csv(file1)

In [90]:
file1 = 'mito_mfn2_combined.csv'
frame_mfn2_f.to_csv(file1)

In [ ]:
#PCA
PCA_DATA = frame_mfn2_f.copy()
PCA_DATA.fillna(0, inplace=True)
features2 = PCA_DATA.columns[:54]  
x2 = PCA_DATA.loc[:, features2].values
y2 = PCA_DATA.loc[:,['group']].values
x2 = StandardScaler().fit_transform(x2)
pd.DataFrame(data = x2, columns = features2).head()
pca2 = PCA(n_components=2)
principalComponents2 = pca2.fit_transform(x2)
PCA_DATA2_COMP = pd.DataFrame(data = principalComponents2
             , columns = ['principal component 1', 'principal component 2'])
importances2 = pd.DataFrame(data = abs(pca2.components_).transpose(), columns = ['PC1', 'PC2'])
importances2['Feature'] = features2
display(importances2.sort_values('PC1', ascending = False).reset_index(drop = True))
imp2 = importances2.sort_values('PC2', ascending = False)[0:30]

ax2 = sns.barplot(x = 'PC2', y = 'Feature', data = imp2)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90)  
plt.show()
    
imp4 = importances2.sort_values('PC1', ascending = False)[0:30]
ax2 = sns.barplot(x = 'PC1', y = 'Feature', data = imp4)

fig = ax2.get_figure()
fig.set_size_inches(4, 8)
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90)  
plt.show()
    
PCA_DATA2_FINAL = pd.concat([PCA_DATA2_COMP, PCA_DATA[['group']]], axis = 1)
PCA_FIG2 = plt.figure(figsize = (8,8))
ax2 = PCA_FIG2.add_subplot(1,1,1) 
ax2.set_xlabel('Principal Component 1', fontsize = 15)
ax2.set_ylabel('Principal Component 2', fontsize = 15)
ax2.set_title('2 Component PCA', fontsize = 20)
ax2.tick_params(axis='x', labelsize=6)
ax2.tick_params(axis='y', labelsize=6)
for tick in ax2.get_xticklabels():
    tick.set_fontname("Arial")
for tick in ax2.get_yticklabels():
    tick.set_fontname("Arial")
    
    
    

targets2 = ['HC', 'S']
colors2 = ['b', 'r']
for target, color in zip(targets2,colors2):
    indicesToKeep = PCA_DATA2_FINAL['group'] == target
    ax2.scatter(PCA_DATA2_FINAL.loc[indicesToKeep, 'principal component 1']
               , PCA_DATA2_FINAL.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
    ax2.legend(targets2)
    ax2.grid()

    
print(pca2.explained_variance_ratio_)
plt.savefig(path + '//' + 'mfn2_PCA2.pdf', dpi = 300)
plt.show()
    
    

In [ ]:
#LDA
PCA_DATA_CLEAN = frame_mfn2_f.copy()

x = PCA_DATA_CLEAN.loc[:, features2].values
y = PCA_DATA_CLEAN.loc[:,['group']].values
y = np.array([int('HC' == y_i[0][:2]) for y_i in y])

y[0]=2
x = StandardScaler().fit_transform(x)

lda = LDA(n_components=2)
lda_x = np.array(lda.fit(x, y).transform(x))


importances = pd.DataFrame(data = abs(lda.scalings_), columns = ['PC1', 'PC2'])
importances['Feature'] = features2
display(importances.sort_values('PC1', ascending = False).reset_index(drop = True))
imp = importances.sort_values('PC2', ascending = False)[0:30]
ax1 = sns.barplot(x = 'PC2', y = 'Feature', data = imp)
    
fig = ax1.get_figure()
fig.set_size_inches(4, 8)
plt.rc('ytick', labelsize=10)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation = 90)  
plt.show()
imp2 = importances.sort_values('PC1', ascending = False)[0:30]
ax2 = sns.barplot(x = 'PC1', y = 'Feature', data = imp2)
fig = ax2.get_figure()
fig.set_size_inches(4, 8)
plt.rc('ytick', labelsize=10)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation = 90) 
plt.show()

font = {'family' : 'Arial',
        'size'   : 22}

plt.rc('font', **font)

scale=6
target_names= ['S', 'HC']
plt.figure()
plt.figure(figsize = (8,8))
for c, i, target_name in zip("rb", [0, 1], target_names):
    plt.scatter(lda_x[y == i,0] , lda_x[y == i,1] , c=c, label=target_name)
    plt.legend(target_names*scale,fontsize=12)
    plt.xlabel('LD1', fontsize = 20, family = 'Arial')
    plt.ylabel('LD2', fontsize = 20, family = 'Arial')
    plt.yticks(family = 'Arial', size = 10)
    plt.xticks(family = 'Arial', size= 10)
print(lda.explained_variance_ratio_)            
plt.savefig(path + '\\' + 'mfn2-LDA.pdf', dpi=600 )
plt.show()

In [ ]:
###prepare data for batch

In [ ]:
##drp
input_path = 'noam_may_analysis_fold1/DRP1'
output_path = 'noam_may_analysis_fold1/DRP1/ready_data_drp1'
add_exp_column_to_csv_files(input_path, output_path)

In [ ]:
##vat
input_path = 'noam_may_analysis_fold1/VAT1'
output_path = 'noam_may_analysis_fold1/VAT1/ready_data_VAT1'
add_exp_column_to_csv_files(input_path, output_path)

In [ ]:
##mfn1
input_path = 'noam_may_analysis_fold1/MFN1'
output_path = 'noam_may_analysis_fold1/MFN1/ready_data_MFN1'
add_exp_column_to_csv_files(input_path, output_path)

In [ ]:
##mfn2
input_path = 'noam_may_analysis_fold1/MFN2'
output_path = 'noam_may_analysis_fold1/MFN2/ready_data_MFN2'
add_exp_column_to_csv_files(input_path, output_path)

In [ ]:
add_plate_column('C:/Users/Uri8s/DRP1')

In [ ]:
add_plate_column('C:/Users/MiguelW12/PycharmProjects/noam_may_analysis_fold1/VAT1/ready_data_VAT1')

In [ ]:
add_plate_column('C:/Users/MiguelW12/PycharmProjects/noam_may_analysis_fold1/MFN1/ready_data_MFN1')

In [ ]:
add_plate_column('C:/Users/MiguelW12/PycharmProjects/noam_may_analysis_fold1/MFN2/ready_data_MFN2')

In [ ]:
path = ('Results_mito_markers_ready_with_plate')
if not os.path.exists(path):
    os.mkdir(path)
    print('Output folder created')

In [ ]:
#drp1
all_files = glob.glob('C:/Users/Uri8s/output_drp/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

drp1_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
drp1_df = drp1_df.drop(columns=["Timepoint", "Time [s]", "Plane", "Row", "Column", "Number of Analyzed Fields", "Height [µm]"
                               ])
drp1_df["sample"] = drp1_df["Severity"].astype(str) + drp1_df["Cell ID"].astype(str) + drp1_df["Compound"].astype(str)  
#selected feat
drp1_df = drp1_df[["spots_chanel_4_final - Corrected Spot Intensity - Mean per Well",
"spots_chanel_4_final - Intensity_Spot chanel_4_final Mean - Mean per Well",
"all_cells - chanel_4_Intensity Mean - Mean per Well", "total_ir_chanel_4_normalized",
"modified_spots_chanel_4 - Total Spot Area - Mean per Well",
"all_cells - chanel_4 Profile 2/5  - Mean per Well", "all_cells - chanel_4 SER Spot 1 px - Mean per Well",
"all_cells - chanel_4 Gabor Max 2 px w2 - Mean per Well",
"all_cells - chanel_4 SER Edge 1 px - Mean per Well", "sample", "Severity", "Cell ID", "Compound",
                  "all_cells - chanel_1 Ratio Width to Length - Mean per Well",
                  "all_cells - chanel_1 Roundness - Mean per Well", "plate"]]
#rename features
drp1_df = drp1_df.rename(
     columns={ "spots_chanel_4_final - Corrected Spot Intensity - Mean per Well":"intensity_1",
"spots_chanel_4_final - Intensity_Spot chanel_4_final Mean - Mean per Well":"intensity_2",
"all_cells - chanel_4_Intensity Mean - Mean per Well":"intensity_3",

"total_ir_chanel_4_normalized":"area_1",
"modified_spots_chanel_4 - Total Spot Area - Mean per Well": "area_2",
"all_cells - chanel_4 Profile 2/5  - Mean per Well": "area_3",

"all_cells - chanel_4 SER Spot 1 px - Mean per Well": "texture_1",
"all_cells - chanel_4 Gabor Max 2 px w2 - Mean per Well": "texture_2",
"all_cells - chanel_4 SER Edge 1 px - Mean per Well": "texture_3" 
             })

In [ ]:
drp1_data_for_batch = drp1_df.groupby(['sample', 'EXP', "Severity", "Cell ID", "Compound", "plate"],as_index=False).agg({
                                                    'intensity_1':'mean',
                                                    'intensity_2':'mean',
                                                    'intensity_3':'mean',
                                                    'area_1':'mean', 
                                                    'area_2':'mean',
                                                    'area_3':'mean', 
                                                    'texture_1':'mean',
                                                    'texture_2':'mean', 
                                                    'texture_3':'mean', 
                                                                })
drp1_data_for_batch.columns = [str(col) + '_drp1' for col in drp1_data_for_batch.columns]
drp1_data_for_batch = drp1_data_for_batch.rename(
     columns={"sample_drp1":"sample",
              "EXP_drp1":"EXP",
              "plate_drp1":"plate",
              "Severity_drp1":"severity",
              "Cell ID_drp1":"cell_id",
              "Compound_drp1":"compound"
             })
print(drp1_data_for_batch)
drp1_data_for_batch.to_csv(path + "//" + 'drp1.csv', index=None)

In [ ]:
#mfn1
all_files = glob.glob('noam_may_analysis_fold1/MFN1/ready_data_mfn1/output/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

mfn1_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
mfn1_df = mfn1_df.drop(columns=["Timepoint", "Time [s]", "Plane", "Row", "Column", "Number of Analyzed Fields", "Height [µm]"
                               ])
mfn1_df["sample"] = mfn1_df["Severity"].astype(str) + mfn1_df["Cell ID"].astype(str) + mfn1_df["Compound"].astype(str)  

mfn1_df = mfn1_df[["spots_chanel_4_final - Corrected Spot Intensity - Mean per Well",
"spots_chanel_4_final - Intensity_Spot chanel_4_final Mean - Mean per Well",
"all_cells - chanel_4_Intensity Mean - Mean per Well", "total_ir_chanel_4_normalized",
"modified_spots_chanel_4 - Total Spot Area - Mean per Well",
"all_cells - chanel_4 Profile 2/5  - Mean per Well", "all_cells - chanel_4 SER Spot 1 px - Mean per Well",
"all_cells - chanel_4 Gabor Max 2 px w2 - Mean per Well",
"all_cells - chanel_4 SER Edge 1 px - Mean per Well", "sample", "EXP", "Severity", "Cell ID", "Compound", "plate"]]
#rename features
mfn1_df = mfn1_df.rename(
     columns={ "spots_chanel_4_final - Corrected Spot Intensity - Mean per Well":"intensity_1",
"spots_chanel_4_final - Intensity_Spot chanel_4_final Mean - Mean per Well":"intensity_2",
"all_cells - chanel_4_Intensity Mean - Mean per Well":"intensity_3",

"total_ir_chanel_4_normalized":"area_1",
"modified_spots_chanel_4 - Total Spot Area - Mean per Well": "area_2",
"all_cells - chanel_4 Profile 2/5  - Mean per Well": "area_3",

"all_cells - chanel_4 SER Spot 1 px - Mean per Well": "texture_1",
"all_cells - chanel_4 Gabor Max 2 px w2 - Mean per Well": "texture_2",
"all_cells - chanel_4 SER Edge 1 px - Mean per Well": "texture_3" 
             })

In [ ]:
mfn1_data_for_batch = mfn1_df.groupby(['sample', 'EXP', "Severity", "Cell ID", "Compound", "plate"],as_index=False).agg({
                                                    'intensity_1':'mean',
                                                    'intensity_2':'mean',
                                                    'intensity_3':'mean',
                                                    'area_1':'mean', 
                                                    'area_2':'mean',
                                                    'area_3':'mean', 
                                                    'texture_1':'mean',
                                                    'texture_2':'mean', 
                                                    'texture_3':'mean', 
                                                                })
mfn1_data_for_batch.columns = [str(col) + '_mfn1' for col in mfn1_data_for_batch.columns]
mfn1_data_for_batch = mfn1_data_for_batch.rename(
     columns={"sample_mfn1":"sample",
              "EXP_mfn1":"EXP",
              "plate_mfn1":"plate",
              "Severity_mfn1":"severity",
              "Cell ID_mfn1":"cell_id",
              "Compound_mfn1":"compound"
             })
print(mfn1_data_for_batch)
mfn1_data_for_batch.to_csv(path + "//" + 'mfn1.csv', index=None)

In [ ]:
#mfn2
all_files = glob.glob('noam_may_analysis_fold1/MFN2/ready_data_mfn2/output/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

mfn2_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
mfn2_df = mfn2_df.drop(columns=["Timepoint", "Time [s]", "Plane", "Row", "Column", "Number of Analyzed Fields", "Height [µm]"
                               ])
mfn2_df["sample"] = mfn2_df["Severity"].astype(str) + mfn2_df["Cell ID"].astype(str) + mfn2_df["Compound"].astype(str)  

mfn2_df = mfn2_df[["spots_chanel_4_final - Corrected Spot Intensity - Mean per Well",
"spots_chanel_4_final - Intensity_Spot chanel_4_final Mean - Mean per Well",
"all_cells - chanel_4_Intensity Mean - Mean per Well", "total_ir_chanel_4_normalized",
"modified_spots_chanel_4 - Total Spot Area - Mean per Well",
"all_cells - chanel_4 Profile 2/5  - Mean per Well", "all_cells - chanel_4 SER Spot 1 px - Mean per Well",
"all_cells - chanel_4 Gabor Max 2 px w2 - Mean per Well",
"all_cells - chanel_4 SER Edge 1 px - Mean per Well", "sample", "EXP", "Severity", "Cell ID", "Compound", "plate"]]
#rename features
mfn2_df = mfn2_df.rename(
     columns={ "spots_chanel_4_final - Corrected Spot Intensity - Mean per Well":"intensity_1",
"spots_chanel_4_final - Intensity_Spot chanel_4_final Mean - Mean per Well":"intensity_2",
"all_cells - chanel_4_Intensity Mean - Mean per Well":"intensity_3",

"total_ir_chanel_4_normalized":"area_1",
"modified_spots_chanel_4 - Total Spot Area - Mean per Well": "area_2",
"all_cells - chanel_4 Profile 2/5  - Mean per Well": "area_3",

"all_cells - chanel_4 SER Spot 1 px - Mean per Well": "texture_1",
"all_cells - chanel_4 Gabor Max 2 px w2 - Mean per Well": "texture_2",
"all_cells - chanel_4 SER Edge 1 px - Mean per Well": "texture_3" 
             })

In [ ]:
mfn2_data_for_batch = mfn2_df.groupby(['sample', 'EXP', "Severity", "Cell ID", "Compound", "plate"],as_index=False).agg({
                                                    'intensity_1':'mean',
                                                    'intensity_2':'mean',
                                                    'intensity_3':'mean',
                                                    'area_1':'mean', 
                                                    'area_2':'mean',
                                                    'area_3':'mean', 
                                                    'texture_1':'mean',
                                                    'texture_2':'mean', 
                                                    'texture_3':'mean', 
                                                                })
mfn2_data_for_batch.columns = [str(col) + '_mfn2' for col in mfn2_data_for_batch.columns]
mfn2_data_for_batch = mfn2_data_for_batch.rename(
     columns={"sample_mfn2":"sample",
              "EXP_mfn2":"EXP",
              "plate_mfn2":"plate",
              "Severity_mfn2":"severity",
              "Cell ID_mfn2":"cell_id",
              "Compound_mfn2":"compound"
             })
print(mfn2_data_for_batch)
mfn2_data_for_batch.to_csv(path + "//" + 'mfn2.csv', index=None)

In [ ]:
#vat
all_files = glob.glob('noam_may_analysis_fold1/VAT1/ready_data_vat1/output/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)

    li.append(df)

vat_df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
vat_df = vat_df.drop(columns=["Timepoint", "Time [s]", "Plane", "Row", "Column", "Number of Analyzed Fields", "Height [µm]"
                               ])
vat_df["sample"] = vat_df["Severity"].astype(str) + vat_df["Cell ID"].astype(str) + vat_df["Compound"].astype(str)  

vat_df = vat_df[["spots_chanel_4_final - Corrected Spot Intensity - Mean per Well",
"spots_chanel_4_final - Intensity_Spot chanel_4_final Mean - Mean per Well",
"all_cells - chanel_4_Intensity Mean - Mean per Well", "total_ir_chanel_4_normalized",
"modified_spots_chanel_4 - Total Spot Area - Mean per Well",
"all_cells - chanel_4 Profile 2/5  - Mean per Well", "all_cells - chanel_4 SER Spot 1 px - Mean per Well",
"all_cells - chanel_4 Gabor Max 2 px w2 - Mean per Well",
"all_cells - chanel_4 SER Edge 1 px - Mean per Well", "sample", "EXP", "Severity", "Cell ID", "Compound", "plate"]]
#rename features
vat_df = vat_df.rename(
     columns={ "spots_chanel_4_final - Corrected Spot Intensity - Mean per Well":"intensity_1",
"spots_chanel_4_final - Intensity_Spot chanel_4_final Mean - Mean per Well":"intensity_2",
"all_cells - chanel_4_Intensity Mean - Mean per Well":"intensity_3",

"total_ir_chanel_4_normalized":"area_1",
"modified_spots_chanel_4 - Total Spot Area - Mean per Well": "area_2",
"all_cells - chanel_4 Profile 2/5  - Mean per Well": "area_3",

"all_cells - chanel_4 SER Spot 1 px - Mean per Well": "texture_1",
"all_cells - chanel_4 Gabor Max 2 px w2 - Mean per Well": "texture_2",
"all_cells - chanel_4 SER Edge 1 px - Mean per Well": "texture_3" 
             })


In [ ]:
vat_data_for_batch = vat_df.groupby(['sample', 'EXP', "Severity", "Cell ID", "Compound", "plate"],as_index=False).agg({
                                                    'intensity_1':'mean',
                                                    'intensity_2':'mean',
                                                    'intensity_3':'mean',
                                                    'area_1':'mean', 
                                                    'area_2':'mean',
                                                    'area_3':'mean', 
                                                    'texture_1':'mean',
                                                    'texture_2':'mean', 
                                                    'texture_3':'mean', 
                                                                })
vat_data_for_batch.columns = [str(col) + '_vat' for col in vat_data_for_batch.columns]
vat_data_for_batch = vat_data_for_batch.rename(
     columns={"sample_vat":"sample",
              "EXP_vat":"EXP",
              "plate_vat":"plate",
              "Severity_vat":"severity",
              "Cell ID_vat":"cell_id",
              "Compound_vat":"compound"
             })
print(vat_data_for_batch)
vat_data_for_batch.to_csv(path + "//" + 'vat.csv', index=None)

In [ ]:
### marker downstream analysis 

In [ ]:
## markers integ

In [ ]:
##radar plot

In [ ]:
df_integ = pd.read_csv('all_markers_df_integ.csv')
df_integ.dtypes

In [17]:
# condition mapping for new column
condition_mapping = {'M': 'Mild', 'S': 'Severe', 'P': 'Premanifest', 'H': 'Control'}

#  create the new column
df_integ = create_new_column_based_on_first_letter(df_integ, 'group_with_id', 'group', condition_mapping)
df_integ["index2"] = df_integ["DS"].astype(str) + df_integ["group_with_id"].astype(str)

In [20]:
specific_value_to_remove = 'mdivi1 25uM'
df_integ = remove_rows_containing_specific_value(df_integ, 'group_with_id', specific_value_to_remove)
df_integ = df_integ.groupby("group").mean()
df_integ = df_integ.reset_index()

In [ ]:
df_integ.set_index(["group"], inplace = True,
                            append = True, drop = False)
df_integ = df_integ.drop(columns=["group"])

In [ ]:
#scale
min_max = df_integ.copy()
min_max = min_max.reset_index()
numeric_columns = min_max.columns[min_max.dtypes == 'float64']
group_column = 'group'

# min-max Scaling
scaler = MinMaxScaler()
df_scaled = min_max.copy()
df_scaled[numeric_columns] = scaler.fit_transform(min_max[numeric_columns])
df_scaled.set_index(["group"], inplace = True,
                            append = True, drop = False)
df_scaled = df_scaled.drop(columns=["group"])
df_scaled = df_scaled + 1
df_scaled = df_scaled.drop(columns=["level_0"])
df_scaled = df_scaled.reset_index()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def radar_plot(df, features, group_column, color_mapping=None):


    missing_features = set(features) - set(df.columns)
    if missing_features:
        raise ValueError(f"The following features are not present in the DataFrame: {missing_features}")

    # columns
    selected_columns = [group_column] + features
    df_selected = df[selected_columns]


    if color_mapping:
        color_discrete_map = color_mapping
    else:
        color_discrete_map = None


    dash_styles = ['solid', 'dash', 'dot', 'dashdot']


    fig = go.Figure()
    for i, group in enumerate(df[group_column].unique()):
        group_data = df[df[group_column] == group]
        fig.add_trace(go.Scatterpolar(
            r=group_data[features].mean(),
            theta=features,
            mode='lines+markers',  
            name=f'{group} - Mean',
            line_shape='linear',
            fill='toself',  
            line=dict(color=color_discrete_map[group] if color_discrete_map else None, dash=dash_styles[i % len(dash_styles)]),
            marker=dict(color=color_discrete_map[group] if color_discrete_map else None, size=10, symbol='circle'),
        ))


    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, df[features].max().max()], tickfont=dict(size=12))),
        showlegend=True,
        legend=dict(x=0, y=1), 
        template='plotly', 
       
    )


    fig.show()


    #  features and group column
    features = ['drp1_intensity','mfn1_intensity','mfn2_intensity',
              'vat1_intensity']
    group_column = 'group'

    # colors 
    color_mapping = {'Control': 'blue', 'Severe': 'red',
                    'Mild': 'green', 'Premanifest':'yellow'}

    # run function
    radar_plot(df_scaled, features, group_column, color_mapping)

In [ ]:
########################################################################